<a href="https://colab.research.google.com/github/kjg331/Scrape_Youtube_Video_Info-Comments/blob/main/(2023.09)Scrape_Youtube_Video_Info_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
import requests
import re
import time
import os
import pandas as pd
import json
import sys
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup

# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [ ]:
def get_page_sources_youtube(url):

    browser = driver
    page_sources = []

    # Move to the page
    url = url
    browser.get(url)

    # Scroll down
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")

    # Iterate until end script of the website
    while True:

        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3.0)

        new_page_height = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        if new_page_height == last_page_height:

            break

        else:
            print("Continue")

        last_page_height = new_page_height

        print("End of scrolling")

        page_source = browser.page_source
        page_sources.append(page_source)

    return page_sources

In [ ]:
def get_info(url):

    url = url
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    info = []

    video_name = soup.find('meta', itemprop="name")['content']
    num_views = soup.find('meta', itemprop="interactionCount")['content']
    date_published = soup.find('meta', itemprop="datePublished")['content']

    info.append([video_name, num_views, date_published])

    video_info = pd.DataFrame(index = ["Info"], columns = ['Video_name', 'Num_views', 'Date_published'],
                              data = info).transpose()
    return video_info

def get_Comment_Like(url):

    browser = driver
    comment = []
    comment_date= []
    num_like = []

    # Move to the page
    url = url
    browser.get(url)

    # Scroll down and wait to get the reply

    browser.execute_script("window.scrollTo(0, 800);")
    time.sleep(10)
    browser.execute_script("window.scrollTo(800, 0);")
    time.sleep(5)
    browser.execute_script("window.scrollTo(10, document.documentElement.scrollHeight);")
    time.sleep(5)

    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")

    # Iterate until end script of the website
    while True:

        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3.0)

        new_page_height = browser.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            print("End scrolling")
            print("---"*30)
            print("Start Scrapping")
            break

        else:
            print("Iterating...")

        last_page_height = new_page_height

    # Get the comment
    for comments in browser.find_elements(By.ID, "content-text"):
        comment.append(comments.text)

    # Get year of the comment("~months,weeks,days" will be 2022 and others will be "2022 - ~years" in this code)
    for dates in browser.find_elements(By.XPATH, '//*[@id="header-author"]/yt-formatted-string/a'):
        comment_date.append(dates.text)

    # Get the num of likes
    for likes in browser.find_elements(By.ID, "vote-count-middle"):

        if likes.text:
            num_like.append(likes.text)
        else:
            num_like.append("0")



    browser.quit()
    # Make the dataframe with scrapped data
    video_content = pd.DataFrame({"Comment": comment, "Comment_date": comment_date, "Likes": num_like})
    video_content.Comment_date = video_content.Comment_date.apply(change_year)
    video_content.Likes = video_content.Likes.apply(convert_str_to_number)

    return video_content

def convert_str_to_number(x):

    total_stars = 0
    num_map = {'천': 1000, '만': 10000, 'K':1000, 'M':1000000, 'B':1000000000}

    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)

    return int(total_stars)

def change_year(w):
    w = w.strip()
    w = re.sub(r"\(수정됨\)", " ", w)
    w = re.sub(r"전", " ", w)
    w = re.sub(r"1년", "2022", w)
    w = re.sub(r"2년", "2021", w)
    w = w.strip()
    w = re.sub("([0-9]+)([^0-9]+$)", "2023", w) # Change any other period to 2022
    return w

In [ ]:
url = "https://www.youtube.com/watch?v=MQ4Fu5vRXjM&t=38s&ab_channel=%EC%9B%90%EC%8B%A0"
test_info = get_info(url)
test_df = get_Comment_Like(url)